<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/%E2%80%9CHW2_Junaid_Khalidi_Py_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import psutil
total_memory = psutil.virtual_memory().total / 1e9
print(f"Total RAM: {total_memory:.2f} GB")

Total RAM: 89.63 GB


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
pip install --upgrade --force-reinstall nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: regex
    Found existing installation: regex 2024.11.6
    Uninstalling regex-2024.11.6:
      Successfully uninstalled regex-2024.11.6
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled

In [4]:
import torch
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup
nltk.download('wordnet')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to /root/nltk_data...


##Dataset Generation

In [75]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


<ipython-input-75-dd748172f966>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [76]:
#Ensure that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)

<ipython-input-76-251f533e7f5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = df['star_rating'].astype(int)


In [7]:
df

,star_rating,review_body
0,5,Great product.
1,5,"What's to say about this commodity item except, what have we come to in this world.<br />Having the need to bnuy captured and compressed air. &#60;lol&#62;"
2,5,"Haven't used yet, but I am sure I will like it."
3,1,"Although this was labeled as &#34;new&#34; the one I received clearly had been used. The box had previously been opened., and the shredder was dirty and the bin was partially full of shredded paper. What was worse is that the unit will not work properly. It is not possible to insert the paper bin so as to enable the shredder to run. It will not operate if the bin is not in place, but I could never get the unit to recognize that the paper bin was actually fully inserted. After cleaning everything thoroughly and vacuuming the paper bin area, it worked ONCE! After that I was unable to get it work at all. I returned the unit immediately for a refund. I feel Amazon misrepresented the unit as &#34;new&#34; when clearly it was not."
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,"I can't live anymore whithout my Palm III. But I also hate to carry things around. Enter the Belt Clip Case for PalmPilot. It is extremely light, holds thightly to my belt or jeans pocket and it features a hard plastic screen protection sewed inside. I let my Palm III (inside this case, fortunately) slip and it fell all the way down on a 17 steps stairs. Not a single scratch! I wish it only had an internal pocket for credit and business cards."
2640348,4,"Although the Palm Pilot is thin and compact it may exceed your pocket capacity. With the belt clip you can carry the Palm Pilot, enter and review data at any time. This especially useful for physicians who are on the move constantly. I have found this carrying case to be an excellent accessory."
2640349,4,This book had a lot of great content without being boring or monotonous. I really enjoy Greek mythology and this book summed it up throughly.
2640350,5,"I am teaching a course in Excel and am using the step-by-step books. They are very good. The class is composed of people who are proficient in 1-2-3, and are computer literate, so we're off and running!"


In [77]:
df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df


<ipython-input-77-56b2611b6d1a>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body
0,1,"This item was absolutely terrible. The ink barely showed out of my brother printer. I wanted to return it, but it was too late to return. Never buying again..."
1,1,out from the box there is a blank area when it print.
2,1,"I read ALL the reviews Bad & Good and made my Choice, Wrong Choice!!! I really, Really Love Amazon.com But they really need to Use some of these Catridges for Theirself and Evaluate Them. my Printer was Printing Fine it just needed Ink so I looked these up on Amazon and took a Chance even with more than a few Negative Reviews. I sent some Samples of the Horrible attempt at printing in the Box With The easy return for Refund. Went to WalMart and picked up 2 combo packs of Kodak ink for my Two Printers and Sure Enough it went back to Printing Perfect, Both of the Printers. Well I guess the Fact that Kodak's Ink is so Cheap that It has a tendantcy to push you to see just how cheap you can get it for. Please Folks Save your Time and Frustrations for more Important Troubles in your life, Don't Be Greedy. Buy the Kodak Ink you Won't be Dissapointed. Thanks Amazon, for Your easy Returns, because with out them I would really be up the Creek without a Paddle. I just can't say enough, \\""How much I love Amazon.com\\"" Thanks,"
3,1,This came very sharp and it would not come apart. I would not recommend buying this because it cut my finger and it came made wrong. This broke in one day and I regret buying it. Waste of money. :(
4,1,"I'd give this day timer zero stars if that was an option. This calendar is 2 years old! It does say 2011 in the fine print - but being the sucker buyer I did not realize I'd have to verify if it was current - and I paid almost $20 for this old, useless calendar."
...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUEN PRODUCTO LO RECOMIENDO PARA PERSONAS QUE POSEEN SUS OFICINAS A
249996,5,"I have had Panasonic 3 phones for a long time. Recently I had no phones, internet, or security system. I decided to start with new phones. After reading all of the reviews I decided this was the phone system I needed. They came very fast. Still no phone. The phone company came out and found a broken cable leading into my house. Once this was fixed I instantly had phones, internet, & security system. I wish I had ordered a four phone system as I found another area I would like a phone. So, checked with Amazon and instantly found one. It arrived before scheduled delivery date. I love the \\""talking\\"" feature. I do not bother to answer my phone when it tells me \\""out of area\\"". I am sure my older phones still work!* Love Panasonic & Amazon!."
249997,5,These are great pens to draw with and I like the softness of the brush side and the fine tip on the other.
249998,5,"I do martial arts, Jiu-Jitsu and I like to take notes and this book is sturdy and it can take a beating in the room when I am throwing it around after the instructional to the side before drilling. It is great for taking notes in general and good solid book. You can definitely defend yourself with this thing. If this book can handle my roughness it can handle your basic needs, will buy again."


In [78]:
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

df['sentiment'] = df['star_rating'].apply(map_sentiment)


In [10]:
#train_df, test_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['sentiment'], random_state=42)


In [11]:
pip install gensim

## 2. Word Embeddings

In [12]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import nltk
nltk.download('punkt')
import gensim.downloader as api

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
# Load pretrained Word2Vec model (Google News 300)
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")
# Checking the vector representation of a word
#print(wv["excellent"])  # A 300-dimensional vector

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
#check semantic similarities
result = wv_google.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("King - Man + Woman =", result[0][0])  # Expected output: 'queen'

King - Man + Woman = queen


In [15]:
#similarities between words
similarity = wv_google.similarity("excellent", "outstanding")
print("Similarity between 'excellent' and 'outstanding':", similarity)

Similarity between 'excellent' and 'outstanding': 0.55674857


In [16]:
import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
df["tokens"] = df["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(df["tokens"].head())
#df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0                                                                                                                                                                                                                                                                                                                                                                                                                [This, item, was, absolutely, terrible, ., The, ink, barely, showed, out, of, my, brother, printer, ., I, wanted, to, return, it, ,, but, it, was, too, late, to, return, ., Never, buying, again, ...]
1                                                                                                                                                                                                                                                                                                                                                                                                              

In [17]:
#converting to word embeddings
'''
def get_word2vec_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # 300D vector

# Applying to dataset using the pretrained Word2Vec model
balanced_df["embedding"] = balanced_df["tokens"].apply(lambda x: get_word2vec_embedding(x, wv))'''
import gensim.downloader as api
import numpy as np
import pandas as pd

# Load the pretrained Word2Vec model
#wv = api.load("word2vec-google-news-300")

# Function to get embeddings for each word in a review
def get_word_embeddings(tokens, model):
    return [model[word] for word in tokens if word in model]  # List of word embeddings

# Load dataset (assume it's preprocessed and tokenized)
#df = pd.read_pickle("amazon_reviews.pkl")  # Replace with actual dataset file

# Apply function to dataset
df["word_embeddings"] = df["tokens"].apply(lambda x: get_word_embeddings(x, wv_google))

###(b) Training Word2Vec model on our own datset

In [18]:
# Training my custom Word2Vec model
custom_w2v = Word2Vec(
    sentences=df["tokens"],  # List of tokenized reviews
    vector_size=300,         # Word embedding size
    window=11,               # Context window size
    min_count=10,            # Minimum word frequency threshold
    workers=4                # Use multiple CPU cores for efficiency
)

# Save the trained model
custom_w2v.save("custom_word2vec.model")

# Example: Check words most similar to "excellent"
print(custom_w2v.wv.most_similar("excellent"))

[('outstanding', 0.8639243245124817), ('exceptional', 0.8005495667457581), ('superb', 0.7127473950386047), ('terrific', 0.7106993198394775), ('fantastic', 0.6900885105133057), ('incredible', 0.6887058019638062), ('amazing', 0.6826050877571106), ('good', 0.6370333433151245), ('great', 0.630925714969635), ('wonderful', 0.6221680641174316)]


In [19]:
result_custom = custom_w2v.wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("Custom Word2Vec: King - Man + Woman =", result_custom[0][0])

Custom Word2Vec: King - Man + Woman = queen


In [20]:
similarity_custom = custom_w2v.wv.similarity("excellent", "outstanding")
print("Custom Word2Vec similarity (excellent vs outstanding):", similarity_custom)

Custom Word2Vec similarity (excellent vs outstanding): 0.8639243


**Comparison based on King - Man + Woman on above 2 models :**

Google News Word2Vec : Returns "Queen"

Custom Word2Vec (Amazon Reviews) : Does return "Queen" but the dataset may lacks historical context for other operations.


**Comparison based on Excellent vs Outstanding :**


Google News Word2Vec : High similarity (~0.9)


Custom Word2Vec (Amazon Reviews) :0.86168975 ( lower due to limited data.)


**Based on Context Understanding**

Google News Word2Vec : General knowledge	Biased towards Amazon review words.

Custom Word2Vec (Amazon Reviews): Biased towards Amazon review words.

**Conclusion:-**

Pretrained Word2Vec (Google News 300) performs better for general language tasks

It was trained on a large-scale dataset (Google News corpus).
It encodes rich semantic relationships between words.
It correctly captures King - Man + Woman = Queen.

Custom Word2Vec is more domain-specific (Amazon Reviews)

It is trained on customer reviews, so it understands sentiment-related words well.
But it lacks general world knowledge, making it weaker for analogy tasks.
Example: It may not return "Queen" for King - Man + Woman, but it might capture sentiment-related similarities better (e.g., "great" ∼ "awesome")

**Best Choice?**

If we need general word understanding, use then pretrained Word2Vec.

If we need domain-specific representations (Amazon reviews, legal, medical, etc.),  our own custom trained Word2V on reviews works better

Word2Vec models seems
to encode semantic similarities between words better

In [21]:
print(custom_w2v.wv.most_similar("excellent"))

[('outstanding', 0.8639243245124817), ('exceptional', 0.8005495667457581), ('superb', 0.7127473950386047), ('terrific', 0.7106993198394775), ('fantastic', 0.6900885105133057), ('incredible', 0.6887058019638062), ('amazing', 0.6826050877571106), ('good', 0.6370333433151245), ('great', 0.630925714969635), ('wonderful', 0.6221680641174316)]


### 3.

### Data cleaning and Data Preprocessing

In [79]:
contractions_dict = {
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "how's": "how is",
    "i'd": "I would",
    "i'll": "I will",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "weren't": "were not",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [80]:
#from locale import D_FMT
import pandas as pd
import re
import nltk
from nltk.corpus import words
import re
#from spellchecker import SpellChecker
import pandas as pd
import re


# Function to filter misspelled words
def remove_misspelled_words(text):
    if isinstance(text, str):
        # Split text into words
        words = text.split()
        # Check each word and retain it only if it's correctly spelled
        corrected_words = [word for word in words if word in spell]
        return " ".join(corrected_words)
    return ""

# Download the English words corpus (only required once)
nltk.download('words')

# Get the set of valid English words
english_words = set(words.words())

# Function to filter non-English words from a string
def filter_non_english_words(text):
    if isinstance(text, str):
        # Split text into words, filter non-English words, and rejoin
        filtered_words = [word for word in text.split() if word.lower() in english_words]
        return " ".join(filtered_words)
    return ""

def expand_contractions(text, contractions_map):
    if isinstance(text, str):
        pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions_map.keys()) + r')\b')
        return pattern.sub(lambda x: contractions_map[x.group()], text)
    return text

def preprocess_reviews(df):
    # Step 1: Convert all reviews to lowercase
    df['cleaned_review'] = df['review_body'].str.lower()

    # Step 2: Remove HTML tags and URLs
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else '')
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'http\S+', '', x) if isinstance(x, str) else '')

    # Step 3: Expand contractions (e.g., can't -> can not)
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: contractions.fix(x))
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: expand_contractions(x, contractions_dict))


    # Step 4: Separate concatenated words (e.g., slot...can't -> slot can't)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'(\w)(\'\w)', r'\1 \2', x))  # Insert space between word and contraction

    # Step 5: Replace multiple dots (two or more) with a space
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\.{2,}', ' ', x))

    # Step 6: Replace non-alphabetical characters (except for apostrophes and spaces) with a space
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^\w\s\'-]', ' ', x))  # Keep alphabet, spaces, apostrophes, and hyphens
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^a-zA-Z\s\']', ' ', x))  # Keep only alphabet, spaces, and apostrophes
    # Step 7: Remove extra spaces (e.g., from replacing punctuation)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # Step 8:  Apply the function to remove misspelled words
    df['cleaned_review'] = df['cleaned_review'].apply(filter_non_english_words)
    #df['cleaned_review'] = df['cleaned_review'].apply(remove_misspelled_words)
    return df

def average_length(df, column='review_body'):
    # Replace non-string values with empty string
    df[column] = df[column].fillna('').astype(str)

    # Calculate average length
    lengths = df[column].apply(len)
    return lengths.mean()



# Calculate average lengths before preprocessing
avg_length_before = average_length(df, column='review_body')

# Preprocess the reviews
reduced_df = preprocess_reviews(df)

# Calculate average lengths after preprocessing
avg_length_after = average_length(df, column='cleaned_review')

# Output average lengths before and after
print(f"Average length before Data Cleaning: {avg_length_before}, Average length after Data Cleaning: {avg_length_after}")



#Average length before cleaning: 319.694505
#Average length after cleaning: 260.67447

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Average length before Data Cleaning: 342.972252, Average length after Data Cleaning: 280.349656


## Data Preprocessing

In [24]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download necessary resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to get the WordNet POS tag from the NLTK POS tag
def get_wordnet_pos(tag):
    if tag.startswith('VB'):  # Verbs
        return wordnet.VERB
    elif tag.startswith('JJ'):  # Adjectives
        return wordnet.ADJ
    elif tag.startswith('NN'):  # Nouns
        return wordnet.NOUN
    elif tag.startswith('RB'):  # Adverbs
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    # Tokenize the sentence and POS tag each token
    tokens = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)

    # Lemmatize using POS tags and filter out stopwords
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens if word.lower() not in stop_words
    ]

    # Return the lemmatized sentence as a string
    return ' '.join(lemmatized_words)

# Apply lemmatization to your dataframe column
df['cleaned_review'] = df['cleaned_review'].apply(lemmatize_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [25]:
print(f"Average length before data preprocessing: {avg_length_after}, Average length after data preprocessing: {average_length(reduced_df, column='cleaned_review')}")


Average length before data preprocessing: 280.349656, Average length after data preprocessing: 153.755496


In [26]:
### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
#wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

Computing Average of Embeddings for both models

In [27]:
# Compute embeddings for both models
df["embedding_google_avg"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom_avg"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))

In [81]:
df_3classes = df.copy()
#df = df[df["sentiment"].isin([1, 2])]
df = df[df['sentiment'] != 3]

# Replace sentiment 1 with 0, and sentiment 2 with 1
df['sentiment'] = df['sentiment'].replace({1: 0, 2: 1})
df_3classes['sentiment'] = df_3classes['sentiment'].replace({1: 0, 2: 1,3:2})


<ipython-input-81-dbb6792c40fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace({1: 0, 2: 1})


In [83]:
df_3classes.shape

(250000, 4)

## 3. Simple Models

### Tf-idf feature extraction

In [43]:
### Tf-idf feature extraction
newdf= reduced_df
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,  # Limit to top 5000 terms (optional)
    stop_words='english',  # Remove common stop words
    #ngram_range=(1,2)  # Use unigrams and bigrams
)
# Fit the vectorizer and transform the data
tfidf_features = tfidf_vectorizer.fit_transform(df['cleaned_review'])

In [44]:
tfidf_features

<200000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3221912 stored elements in Compressed Sparse Row format>

In [45]:
# Train - Test Split
from sklearn.model_selection import train_test_split
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google_avg"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom_avg"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_tfidf, X_test_tfidf, _, _ = train_test_split(
    tfidf_features, df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

In [46]:
print(type(X_train_google))  # <class 'list'>
print(type(X_train_google[0]))  # <class 'numpy.ndarray'>
print(X_train_google[0].shape)  # (300,)

<class 'list'>
<class 'numpy.ndarray'>
(300,)


In [47]:
print(type(X_train_custom))  # <class 'list'>
print(type(X_train_custom[0]))  # <class 'numpy.ndarray'>
print(X_train_custom[0].shape)  # (300,)

<class 'list'>
<class 'numpy.ndarray'>
(300,)


<ipython-input-37-c4a1f248d9fb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace({1: 0, 2: 1})


In [41]:
df_3classes.shape

(250000, 8)

In [48]:
#Perceptron training
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred, average="binary")
    train_recall = recall_score(y_train, y_train_pred, average="binary")
    train_f1 = f1_score(y_train, y_train_pred, average="binary")

    # Test metrics
    test_acc = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average="binary")
    test_recall = recall_score(y_test, y_test_pred, average="binary")
    test_f1 = f1_score(y_test, y_test_pred, average="binary")

    # Print results
    print(f"\n📌 {model_name} Results:")
    print(f"✅ Training Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")
    print(f"✅ Training Precision: {train_precision:.4f} | Test Precision: {test_precision:.4f}")
    print(f"✅ Training Recall: {train_recall:.4f} | Test Recall: {test_recall:.4f}")
    print(f"✅ Training F1 Score: {train_f1:.4f} | Test F1 Score: {test_f1:.4f}")

    #return {
    #    "train_acc": train_acc, "test_acc": test_acc,
    #    "train_precision": train_precision, "test_precision": test_precision,
    #    "train_recall": train_recall, "test_recall": test_recall,
    #    "train_f1": train_f1, "test_f1": test_f1
    #}

### **1️⃣ Perceptron Models**
print("\n🔹 Training Perceptron Models...")
perc_google = Perceptron()
perc_google.fit(X_train_google, y_train)
evaluate_model(perc_google, X_train_google, X_test_google, y_train, y_test, "Perceptron (Google News W2V)")

perc_custom = Perceptron()
perc_custom.fit(X_train_custom, y_train)
evaluate_model(perc_custom, X_train_custom, X_test_custom, y_train, y_test, "Perceptron (Custom W2V)")

perc_tfidf = Perceptron()
perc_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(perc_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "Perceptron (TF-IDF)")


### **2️⃣ SVM Models**
print("\n🔹 Training SVM Models...")
svm_google = LinearSVC(random_state=42, max_iter=10000)
svm_google.fit(X_train_google, y_train)
evaluate_model(svm_google, X_train_google, X_test_google, y_train, y_test, "SVM (Google News W2V)")

svm_custom = LinearSVC(random_state=42, max_iter=10000)
svm_custom.fit(X_train_custom, y_train)
evaluate_model(svm_custom, X_train_custom, X_test_custom, y_train, y_test, "SVM (Custom W2V)")

svm_tfidf =LinearSVC(random_state=42, max_iter=10000)
svm_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(svm_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "SVM (TF-IDF)")



🔹 Training Perceptron Models...

📌 Perceptron (Google News W2V) Results:
✅ Training Accuracy: 0.7853 | Test Accuracy: 0.7836
✅ Training Precision: 0.7789 | Test Precision: 0.7784
✅ Training Recall: 0.7969 | Test Recall: 0.7929
✅ Training F1 Score: 0.7878 | Test F1 Score: 0.7856

📌 Perceptron (Custom W2V) Results:
✅ Training Accuracy: 0.7325 | Test Accuracy: 0.7310
✅ Training Precision: 0.8617 | Test Precision: 0.8577
✅ Training Recall: 0.5538 | Test Recall: 0.5540
✅ Training F1 Score: 0.6743 | Test F1 Score: 0.6731

📌 Perceptron (TF-IDF) Results:
✅ Training Accuracy: 0.8109 | Test Accuracy: 0.7904
✅ Training Precision: 0.8219 | Test Precision: 0.8001
✅ Training Recall: 0.7937 | Test Recall: 0.7742
✅ Training F1 Score: 0.8076 | Test F1 Score: 0.7870

🔹 Training SVM Models...

📌 SVM (Google News W2V) Results:
✅ Training Accuracy: 0.8122 | Test Accuracy: 0.8125
✅ Training Precision: 0.7951 | Test Precision: 0.7959
✅ Training Recall: 0.8412 | Test Recall: 0.8407
✅ Training F1 Score: 0.817

### TF-IDF remains the best performing feature type for both Perceptron and SVM, achieving: Best test accuracy (84.1%) with SVM.

### SVM outperforms Perceptron consistently.

### Custom Word2Vec benefits from domain-specific training but still lags behind TF-IDF.

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


## 4. Feed Forward Neural Networks Using Pytorch

In [50]:

# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google_avg"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom_avg"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

#For ternary , calculating the embeddings:
df_3classes["embedding_google_avg"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df_3classes["embedding_custom_avg"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


# Train-test split for Ternary (80% training, 20% testing)
X_train_google_ternary, X_test_google_ternary, y_train_ter, y_test_ter = train_test_split(
    list(df_3classes["embedding_google_avg"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)

X_train_custom_ternary, X_test_custom_ternary, _, _ = train_test_split(
    list(df_3classes["embedding_custom_avg"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)


In [51]:
y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))

In [52]:
y_train_ternary = np.array(y_train_ter .replace({1: 0, 2: 1,3:2}))
y_test_ternary = np.array(y_test_ter.replace({1: 0, 2: 1,3:2}))

In [53]:
#creating a Feedforward MLP with two hidden layers (50 & 10 nodes).
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)  # First hidden layer (50 neurons)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)  # Second hidden layer (10 neurons)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(10, num_classes)  # Output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [54]:
def train_model(X_train, y_train, X_test, y_test, num_classes, epochs=250, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Convert numpy arrays to PyTorch tensors and move to GPU
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.long).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test, dtype=torch.long).to(device)

    # Define model and move to GPU
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MLP(input_size=300, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluate on test set
    with torch.no_grad():
        test_outputs = model(X_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = accuracy_score(y_test.cpu().numpy(), predicted.cpu().numpy())

    return accuracy



## Printing results of Binary Classification using
###    i) Pretrained Google Average Word Embeddings
###    ii) Custom Averaged Word Embeddings

In [55]:
# Training Binary Classification Models:
print("Binary Classification (Positive vs Negative)")

# Train on Google Word2Vec
acc_binary_google = train_model(X_train_google, y_train_binary, X_test_google, y_test_binary, num_classes=2)
print()
print(f"✅ Google W2V Accuracy (Binary): {acc_binary_google:.4f}")
print()
# Train on Custom Word2Vec
acc_binary_custom = train_model(X_train_custom, y_train_binary, X_test_custom, y_test_binary, num_classes=2)
print(f"✅ Custom W2V Accuracy (Binary): {acc_binary_custom:.4f}")




Binary Classification (Positive vs Negative)


<ipython-input-54-cb1ffaa53ced>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X_train = torch.tensor(X_train, dtype=torch.float32).to(device)


Epoch [5/250], Loss: 0.6918
Epoch [10/250], Loss: 0.6848
Epoch [15/250], Loss: 0.6757
Epoch [20/250], Loss: 0.6638
Epoch [25/250], Loss: 0.6489
Epoch [30/250], Loss: 0.6306
Epoch [35/250], Loss: 0.6101
Epoch [40/250], Loss: 0.5887
Epoch [45/250], Loss: 0.5673
Epoch [50/250], Loss: 0.5472
Epoch [55/250], Loss: 0.5289
Epoch [60/250], Loss: 0.5130
Epoch [65/250], Loss: 0.4996
Epoch [70/250], Loss: 0.4885
Epoch [75/250], Loss: 0.4792
Epoch [80/250], Loss: 0.4714
Epoch [85/250], Loss: 0.4648
Epoch [90/250], Loss: 0.4592
Epoch [95/250], Loss: 0.4544
Epoch [100/250], Loss: 0.4503
Epoch [105/250], Loss: 0.4467
Epoch [110/250], Loss: 0.4435
Epoch [115/250], Loss: 0.4406
Epoch [120/250], Loss: 0.4381
Epoch [125/250], Loss: 0.4357
Epoch [130/250], Loss: 0.4336
Epoch [135/250], Loss: 0.4317
Epoch [140/250], Loss: 0.4299
Epoch [145/250], Loss: 0.4283
Epoch [150/250], Loss: 0.4268
Epoch [155/250], Loss: 0.4254
Epoch [160/250], Loss: 0.4241
Epoch [165/250], Loss: 0.4229
Epoch [170/250], Loss: 0.4218


## Printing results of Ternary Classification using
###    i) Pretrained Google Average Word Embeddings
###    ii) Custom Averaged Word Embeddings

In [56]:
print("Ternary Classification (Positive, Negative, Neutral)")
#X_train_google_ternary, X_test_google_ternary
#X_train_custom_ternary, X_test_custom_ternary
# Train on Google Word2Vec
acc_ternary_google = train_model(X_train_google_ternary, y_train_ternary, X_test_google_ternary, y_test_ternary, num_classes=3)
print(f"✅ Google W2V Accuracy (Ternary): {acc_ternary_google:.4f}")

# Train on Custom Word2Vec
acc_ternary_custom = train_model(X_train_custom_ternary, y_train_ternary, X_test_custom_ternary, y_test_ternary, num_classes=3)
print(f"✅ Custom W2V Accuracy (Ternary): {acc_ternary_custom:.4f}")


Ternary Classification (Positive, Negative, Neutral)
Epoch [5/250], Loss: 1.1195
Epoch [10/250], Loss: 1.1106
Epoch [15/250], Loss: 1.0993
Epoch [20/250], Loss: 1.0860
Epoch [25/250], Loss: 1.0705
Epoch [30/250], Loss: 1.0525
Epoch [35/250], Loss: 1.0316
Epoch [40/250], Loss: 1.0125
Epoch [45/250], Loss: 0.9970
Epoch [50/250], Loss: 0.9836
Epoch [55/250], Loss: 0.9700
Epoch [60/250], Loss: 0.9557
Epoch [65/250], Loss: 0.9413
Epoch [70/250], Loss: 0.9280
Epoch [75/250], Loss: 0.9158
Epoch [80/250], Loss: 0.9046
Epoch [85/250], Loss: 0.8947
Epoch [90/250], Loss: 0.8859
Epoch [95/250], Loss: 0.8782
Epoch [100/250], Loss: 0.8713
Epoch [105/250], Loss: 0.8650
Epoch [110/250], Loss: 0.8595
Epoch [115/250], Loss: 0.8541
Epoch [120/250], Loss: 0.8495
Epoch [125/250], Loss: 0.8452
Epoch [130/250], Loss: 0.8413
Epoch [135/250], Loss: 0.8377
Epoch [140/250], Loss: 0.8342
Epoch [145/250], Loss: 0.8309
Epoch [150/250], Loss: 0.8275
Epoch [155/250], Loss: 0.8243
Epoch [160/250], Loss: 0.8213
Epoch [



 # b)
 ###  concatenate the first 10 Word2Vec vectors for each review as the input feature (x = [WT1, ..., WT10]) and train the neural network

In [84]:
def concat_w2v_vectors(tokens, model, num_vectors=10):
    # Use model[word] directly for KeyedVectors (Google W2V) and model.wv[word] for Word2Vec models
    vectors = [model[word] if isinstance(model, gensim.models.KeyedVectors) else model.wv[word] for word in tokens if word in model]

    # If fewer than num_vectors words exist, pad with zero vectors
    while len(vectors) < num_vectors:
        vectors.append(np.zeros(model.vector_size))

    # Take only the first num_vectors
    vectors = vectors[:num_vectors]

    # Flatten (concatenate) the vectors
    return np.concatenate(vectors, axis=0)

# Convert dataset to concatenated vectors
X_google_concat = np.array([concat_w2v_vectors(tokens, wv_google) for tokens in df["cleaned_review"]])
X_custom_concat = np.array([concat_w2v_vectors(tokens, custom_w2v.wv) for tokens in df["cleaned_review"]])


In [85]:
print("Training Data Shape:", X_google_concat.shape)

Training Data Shape: (200000, 3000)


In [ ]:
#print("Ternary training Data Shape:", X_google_concat_tern.shape)

In [86]:
X_google_concat = X_google_concat.astype(np.float32)
X_custom_concat = X_custom_concat.astype(np.float32)



In [87]:
from sklearn.model_selection import train_test_split

# Binary classification split (one at a time)
X_train_google_concat_binary, X_test_google_concat_binary, y_train_google_concat_binary, y_test_google_concat_binary = train_test_split(
    X_google_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

#del X_google_concat  # Free memory

X_train_custom_concat_binary, X_test_custom_concat_binary, y_train_custom_concat_binary, y_test_custom_concat_binary = train_test_split(
    X_custom_concat, df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

#del X_custom_concat  # Free memory

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [97]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score

def train_model_concat(X_train, y_train, X_test, y_test, num_classes, epochs=250, lr=0.005):
    # ✅ Move tensors to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ✅ Convert Pandas Series to NumPy before torch.tensor()
    if num_classes == 2:
        y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
        y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))
        X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_binary, dtype=torch.long)
        X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_binary, dtype=torch.long)

    elif num_classes == 3:
        y_train_ternary = np.array(y_train.replace({1: 0, 2: 1, 3: 2}))  # ✅ Fix variable name
        y_test_ternary = np.array(y_test.replace({1: 0, 2: 1, 3: 2}))  # ✅ Fix variable name
        X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train_ternary, dtype=torch.long)
        X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test_ternary, dtype=torch.long)


    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)
    # ✅ Define the model first
    model = MLP(input_size=3000, num_classes=num_classes)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # ✅ Move input tensors to the same device as the model
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluation
    with torch.no_grad():
        train_outputs = model(X_train)
        _, train_predicted = torch.max(train_outputs, 1)
        train_accuracy = accuracy_score(y_train.cpu().numpy(), train_predicted.cpu().numpy())  # ✅ Move to CPU before converting to NumPy

        test_outputs = model(X_test)
        _, test_predicted = torch.max(test_outputs, 1)
        test_accuracy = accuracy_score(y_test.cpu().numpy(), test_predicted.cpu().numpy())  # ✅ Move to CPU before converting to NumPy

    return train_accuracy, test_accuracy

In [90]:
# Binary Classification (Concatenated Word2Vec)
print("Binary Classification (Concatenated Word2Vec)")

# Google Word2Vec
train_acc_google_concat, test_acc_google_concat = train_model_concat(
    X_train_google_concat_binary, y_train_google_concat_binary,
    X_test_google_concat_binary, y_test_google_concat_binary,
    num_classes=2
)
print(f"✅  Google W2V - Train Accuracy (Binary - Concat): {train_acc_google_concat:.4f}")
print(f"✅  Google W2V - Test Accuracy (Binary - Concat): {test_acc_google_concat:.4f}")

# Custom Word2Vec
train_acc_custom_concat, test_acc_custom_concat = train_model_concat(
    X_train_custom_concat_binary, y_train_custom_concat_binary,
    X_test_custom_concat_binary, y_test_custom_concat_binary,
    num_classes=2
)
print(f"✅ Custom W2V - Train Accuracy (Binary - Concat): {train_acc_custom_concat:.4f}")
print(f"✅ Custom W2V - Test Accuracy (Binary - Concat): {test_acc_custom_concat:.4f}")

Binary Classification (Concatenated Word2Vec)
Epoch [5/160], Loss: 0.6779
Epoch [10/160], Loss: 0.6534
Epoch [15/160], Loss: 0.6395
Epoch [20/160], Loss: 0.6304
Epoch [25/160], Loss: 0.6240
Epoch [30/160], Loss: 0.6180
Epoch [35/160], Loss: 0.6126
Epoch [40/160], Loss: 0.6075
Epoch [45/160], Loss: 0.6024
Epoch [50/160], Loss: 0.5974
Epoch [55/160], Loss: 0.5922
Epoch [60/160], Loss: 0.5874
Epoch [65/160], Loss: 0.5829
Epoch [70/160], Loss: 0.5782
Epoch [75/160], Loss: 0.5737
Epoch [80/160], Loss: 0.5700
Epoch [85/160], Loss: 0.5665
Epoch [90/160], Loss: 0.5649
Epoch [95/160], Loss: 0.5629
Epoch [100/160], Loss: 0.5626
Epoch [105/160], Loss: 0.5623
Epoch [110/160], Loss: 0.5563
Epoch [115/160], Loss: 0.5530
Epoch [120/160], Loss: 0.5510
Epoch [125/160], Loss: 0.5483
Epoch [130/160], Loss: 0.5461
Epoch [135/160], Loss: 0.5442
Epoch [140/160], Loss: 0.5446
Epoch [145/160], Loss: 0.5419
Epoch [150/160], Loss: 0.5428
Epoch [155/160], Loss: 0.5401
Epoch [160/160], Loss: 0.5380
✅  Google W2V 

## CONCLUSIONS:
 The pretrained Google W2V model (81.57%) and Custom W2V model (84.56%) achieve accuracy values comparable to or slightly better than the SVM models

 The Custom W2V (84.56%) performs better than the best simple model, suggesting that a well-trained word embedding can boost performance.

The Google W2V (81.57%) performs similar to SVM (Google W2V) (81.25%), indicating that SVM can efficiently use pretrained embeddings.

The SVM (TF-IDF) model achieves 84.08%, which is almost equal to Custom W2V (84.56%).

Pretrained word embeddings (especially Custom W2V) provide a slight advantage over basic ML models but are not always significantly better than well-tuned TF-IDF

If computational efficiency is a concern, SVM with TF-IDF is a strong alternative to using pretrained embeddings.


In [91]:
del X_google_concat
del X_custom_concat
del X_train_google_concat_binary
del X_test_google_concat_binary
del y_train_google_concat_binary
del y_test_google_concat_binary
del X_train_custom_concat_binary
del X_test_custom_concat_binary
del y_train_custom_concat_binary
del y_test_custom_concat_binary
#Similarly for Ternary , get the 3000 input vector size:
# Convert dataset to concatenated vectors
#X_google_concat_tern = X_google_concat_tern.astype(np.float32)
#X_custom_concat_tern = X_custom_concat_tern.astype(np.float32)
X_google_concat_tern = np.array([concat_w2v_vectors(tokens, wv_google) for tokens in df_3classes["cleaned_review"]])
X_custom_concat_tern = np.array([concat_w2v_vectors(tokens, custom_w2v.wv) for tokens in df_3classes["cleaned_review"]])


In [92]:
print(df_3classes['sentiment'].unique())
#df_3classes['sentiment'] = df_3classes['sentiment'].replace({1: 0, 2: 1, 3: 2})
print(df_3classes['sentiment'].unique())


[1 2 0]
[1 2 0]


In [93]:
# Ternary classification split

X_train_google_concat_ternary, X_test_google_concat_ternary, y_train_google_concat_ternary, y_test_google_concat_ternary = train_test_split(
    X_google_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])


X_train_custom_concat_ternary, X_test_custom_concat_ternary, y_train_custom_concat_ternary, y_test_custom_concat_ternary = train_test_split(
    X_custom_concat_tern, df_3classes['sentiment'], test_size=0.2, random_state=42, stratify=df_3classes['sentiment'])



In [94]:
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))

CUDA Available: True
GPU Name: NVIDIA A100-SXM4-40GB


In [98]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
# ternary Classification (Concatenated Word2Vec)
print("Ternary Classification (Concatenated Word2Vec)")

# Google Word2Vec
train_acc_google_concat_tern, test_acc_google_concat_tern = train_model_concat(
    X_train_google_concat_ternary, y_train_google_concat_ternary,
    X_test_google_concat_ternary, y_test_google_concat_ternary,
    num_classes=3
)
print(f"✅  Google W2V - Train Accuracy (Ternary - Concat): {train_acc_google_concat_tern:.4f}")
print(f"✅  Google W2V - Test Accuracy (Ternary - Concat): {test_acc_google_concat_tern:.4f}")

# Custom Word2Vec
train_acc_custom_concat_tern, test_acc_custom_concat_tern = train_model_concat(
    X_train_custom_concat_ternary, y_train_custom_concat_ternary,
    X_test_custom_concat_ternary, y_test_custom_concat_ternary,
    num_classes=3
)
print(f"✅ Custom W2V - Train Accuracy (Ternary - Concat): {train_acc_custom_concat_tern:.4f}")
print(f"✅ Custom W2V - Test Accuracy (Ternary - Concat): {test_acc_custom_concat_tern:.4f}")

Ternary Classification (Concatenated Word2Vec)
Epoch [5/250], Loss: 0.6016
Epoch [10/250], Loss: 0.5128
Epoch [15/250], Loss: 0.5242
Epoch [20/250], Loss: 0.5274
Epoch [25/250], Loss: 0.5033
Epoch [30/250], Loss: 0.5022
Epoch [35/250], Loss: 0.5053
Epoch [40/250], Loss: 0.5011
Epoch [45/250], Loss: 0.4991
Epoch [50/250], Loss: 0.4995
Epoch [55/250], Loss: 0.4992
Epoch [60/250], Loss: 0.4984
Epoch [65/250], Loss: 0.4979
Epoch [70/250], Loss: 0.4976
Epoch [75/250], Loss: 0.4974
Epoch [80/250], Loss: 0.4972
Epoch [85/250], Loss: 0.4970
Epoch [90/250], Loss: 0.4967
Epoch [95/250], Loss: 0.4965
Epoch [100/250], Loss: 0.4963
Epoch [105/250], Loss: 0.4961
Epoch [110/250], Loss: 0.4959
Epoch [115/250], Loss: 0.4957
Epoch [120/250], Loss: 0.4956
Epoch [125/250], Loss: 0.4954
Epoch [130/250], Loss: 0.4953
Epoch [135/250], Loss: 0.4951
Epoch [140/250], Loss: 0.4950
Epoch [145/250], Loss: 0.4948
Epoch [150/250], Loss: 0.4947
Epoch [155/250], Loss: 0.4946
Epoch [160/250], Loss: 0.4944
Epoch [165/25

# 5. CNN (Using Pytorch):

In [ ]:
df.shape

## Binary Classification using Pytorch :

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ast

# ✅ Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# ✅ Set parameters
max_review_length = 50  # Fixed review length (truncate/pad)
embedding_dim = 300     # Word embedding dimension
batch_size = 64
num_epochs = 10
learning_rate = 0.001


# ✅ Function to safely convert embeddings
def safe_convert(embedding):
    try:
        return np.array(ast.literal_eval(embedding), dtype=np.float32)  # Safer parsing
    except (ValueError, SyntaxError):
        return np.zeros((300,), dtype=np.float32)  # Handle corrupted data (return a 1D array of shape (300,))

# ✅ Convert 'embedding_google' column to NumPy array
X = np.array(df["embedding_google"].apply(safe_convert).tolist(), dtype=np.float32)
y = df["sentiment"].values  # Assuming 'sentiment' is the label column

# Initialize X_reshaped to have the right shape: (num_samples, 50, 300)
X_reshaped = np.zeros((len(X), max_review_length, embedding_dim), dtype=np.float32)

# Reshape each review to have a fixed length of 50 words, ensuring each embedding has shape (300,)
for i, review in enumerate(X):
    review_length = len(review)

    # Truncate longer reviews
    if review_length > max_review_length:
        X_reshaped[i] = review[:max_review_length]
    else:
        # Pad shorter reviews with zero vectors (each padding element is a vector of shape (300,))
        X_reshaped[i, :review_length] = review

# Verify shapes
print("X shape after reshaping:", X_reshaped.shape)  # Expected: (200000, 50, 300)
print("y shape:", y.shape)  # Expected: (200000,)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Convert to PyTorch tensors and move to GPU
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train - 1, dtype=torch.long).to(device)  # Adjust labels to (0,1,2)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test - 1, dtype=torch.long).to(device)

# ✅ Add the missing channel dimension for CNN input
X_train_tensor = X_train_tensor.unsqueeze(1)  # Shape: (num_samples, 1, 50, 300)
X_test_tensor = X_test_tensor.unsqueeze(1)

# ✅ Print final tensor shapes
print("Final X_train_tensor shape:", X_train_tensor.shape)  # Expected: (num_samples, 1, 50, 300)
print("Final X_test_tensor shape:", X_test_tensor.shape)

# ✅ Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ✅ Define CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SentimentCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=50, kernel_size=(3, 3), padding=1)  # Output: (50, 50, 300)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  # Output: (50, 25, 150)
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size=(3, 3), padding=1)  # Output: (10, 25, 150)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)  # Output: (10, 12, 75)

        # Compute the flattened feature size
        self.fc = nn.Linear(10 * 12 * 75, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# ✅ Train and Evaluate Model
def train_cnn_model(num_classes=2):
    model = SentimentCNN(num_classes=num_classes).to(device)  # Move model to GPU
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move batch to GPU
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    # Evaluate Model
    model.eval()
    with torch.no_grad():
        test_preds = torch.cat([torch.argmax(model(X_batch.to(device)), dim=1) for X_batch, _ in test_loader])
        test_acc = accuracy_score(y_test_tensor.cpu().numpy(), test_preds.cpu().numpy())

    print(f"Test Accuracy: {test_acc:.4f}")
    return model

# ✅ Train model for binary classification
print("Training CNN for Binary Classification...")
binary_model = train_cnn_model(num_classes=2)

# ✅ Train model for ternary classification
print("Training CNN for Ternary Classification...")
ternary_model = train_cnn_model(num_classes=3)

In [ ]:
# Trial and error for CNN:

In [1]:
import gensim.downloader as api
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Load the Google News Word2Vec model (300D embeddings)
#wv_google
wv_google = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


<ipython-input-4-dd748172f966>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [5]:
# Ensuring that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

df['sentiment'] = df['star_rating'].apply(map_sentiment)
df

<ipython-input-5-6d3d452f7c73>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = df['star_rating'].astype(int)


,star_rating,review_body,sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
3,1,Although this was labeled as &#34;new&#34; the...,2
4,4,Gorgeous colors and easy to use,1
...,...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...,1
2640348,4,Although the Palm Pilot is thin and compact it...,1
2640349,4,This book had a lot of great content without b...,1
2640350,5,I am teaching a course in Excel and am using t...,1


In [6]:
df=df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df

<ipython-input-6-238ddc82b0f8>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df=df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body,sentiment
0,1,This item was absolutely terrible. The ink bar...,2
1,1,out from the box there is a blank area when it...,2
2,1,I read ALL the reviews Bad & Good and made my ...,2
3,1,This came very sharp and it would not come apa...,2
4,1,I'd give this day timer zero stars if that was...,2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,1
249996,5,I have had Panasonic 3 phones for a long time....,1
249997,5,These are great pens to draw with and I like t...,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",1


In [7]:

import nltk
nltk.download('punkt_tab')
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize words
    return [word for word in tokens if word in wv_google]  # Keep only words in Word2Vec vocabulary

# Apply preprocessing to all reviews
df["cleaned_review"] = df["review_body"].apply(preprocess_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [8]:
def get_word_embeddings(words, model):
     return [model[word] for word in words if word in model]  # Get embeddings for each word

#Apply function to dataset
df["word_embeddings2"] = df["cleaned_review"].apply(lambda x: get_word_embeddings(x, wv_google))


In [9]:
df_3classes = df.copy()
#df = df[df["sentiment"].isin([1, 2])]
df = df[df['sentiment'] != 3]

# Replace sentiment 1 with 0, and sentiment 2 with 1
df['sentiment'] = df['sentiment'].replace({1: 0, 2: 1})
df_3classes['sentiment'] = df_3classes['sentiment'].replace({1: 0, 2: 1,3:2})


<ipython-input-9-dbb6792c40fa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace({1: 0, 2: 1})


In [10]:
df_3classes["word_embeddings2"] = df_3classes["cleaned_review"].apply(lambda x: get_word_embeddings(x, wv_google))


In [11]:
print(df_3classes["sentiment"].unique())
print(df.shape)
print(df_3classes.shape)

[1 2 0]
(200000, 5)
(250000, 5)


In [15]:
import numpy as np
print(np.array(df["word_embeddings2"].iloc[0]).shape)


(26, 300)


In [ ]:
print(type(df["word_embeddings2"].iloc[0]))

In [14]:
df


,star_rating,review_body,sentiment,cleaned_review,word_embeddings2
0,1,This item was absolutely terrible. The ink bar...,1,"[this, item, was, absolutely, terrible, the, i...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
1,1,out from the box there is a blank area when it...,1,"[out, from, the, box, there, is, blank, area, ...","[[0.110839844, -0.15820312, 0.020751953, 0.166..."
2,1,I read ALL the reviews Bad & Good and made my ...,1,"[i, read, all, the, reviews, bad, good, made, ...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
3,1,This came very sharp and it would not come apa...,1,"[this, came, very, sharp, it, would, not, come...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
4,1,I'd give this day timer zero stars if that was...,1,"[id, give, this, day, timer, zero, stars, if, ...","[[0.29492188, -0.12109375, 0.0006828308, 0.220..."
...,...,...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,0,"[lo, que, para, mi, oficina, en, casa, muy, bu...","[[0.13769531, -0.023803711, 0.029663086, 0.197..."
249996,5,I have had Panasonic 3 phones for a long time....,0,"[i, have, had, panasonic, phones, for, long, t...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
249997,5,These are great pens to draw with and I like t...,0,"[these, are, great, pens, draw, with, i, like,...","[[-0.016235352, 0.09423828, 0.091796875, 0.119..."
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",0,"[i, do, martial, arts, jiujitsu, i, like, take...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."


In [13]:
df.shape

(200000, 5)

In [ ]:
del word

In [17]:
import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    if len(embeddings) >= max_length:
        return np.array(embeddings[:max_length])  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Apply padding/truncation
df["padded_embeddings"] = df["word_embeddings2"].apply(lambda x: pad_or_truncate(x, max_review_length, embedding_dim))

# Convert to NumPy array
#X = np.stack(df["padded_embeddings"].values)  # Shape: (num_samples, 50, 300)

<ipython-input-17-208670106c03>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["padded_embeddings"] = df["word_embeddings2"].apply(lambda x: pad_or_truncate(x, max_review_length, embedding_dim))


In [18]:
del wv_google

In [20]:
df = df[["padded_embeddings", "sentiment"]]

# Force garbage collection to free memory
import gc
gc.collect()


98

In [21]:
df

,padded_embeddings,sentiment
0,"[[0.109375, 0.140625, -0.03173828125, 0.166015...",1
1,"[[0.11083984375, -0.158203125, 0.020751953125,...",1
2,"[[-0.22558594, -0.01953125, 0.09082031, 0.2373...",1
3,"[[0.109375, 0.140625, -0.03173828125, 0.166015...",1
4,"[[0.294921875, -0.12109375, 0.0006828308105468...",1
...,...,...
249995,"[[0.1376953125, -0.0238037109375, 0.0296630859...",0
249996,"[[-0.22558594, -0.01953125, 0.09082031, 0.2373...",0
249997,"[[-0.0162353515625, 0.09423828125, 0.091796875...",0
249998,"[[-0.22558594, -0.01953125, 0.09082031, 0.2373...",0


In [22]:
import numpy as np
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df['padded_embeddings'] contains the padded word embeddings of size (50, 300)
# Let's also assume that df['label'] contains the binary sentiment labels (1 or 2)

# Define a custom Dataset class to handle large datasets more efficiently
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

# Convert to numpy arrays
X = np.stack(df['padded_embeddings'].values)  # Shape: (200000, 50, 300)
y = df['sentiment'].values #- 1  # Subtract 1 to adjust labels to start from 0 (for PyTorch)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Dataset instances
train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

# Create DataLoader for batching with smaller batch size
batch_size = 16  # Decrease the batch size to reduce memory usage
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Using device: cuda


In [23]:
# Define the CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, output_dim=2):
        super(SentimentCNN, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=300, out_channels=50, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.relu1 = nn.ReLU()

        # Second Convolutional Layer
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.relu2 = nn.ReLU()

        # Calculate the size of flattened features after the convolution layers
        self.flat_features = 10 * 12  # After two max pooling operations: 50 -> 25 -> 12

        # Fully Connected Layer
        self.fc = nn.Linear(self.flat_features, output_dim)

    def forward(self, x):
        # Input shape: (batch_size, seq_length, embedding_dim)
        # Transpose to: (batch_size, embedding_dim, seq_length)
        x = x.transpose(1, 2)

        # First conv layer
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        # Second conv layer
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        # Flatten
        x = x.view(-1, self.flat_features)

        # Fully connected layer
        x = self.fc(x)
        return x

In [24]:
# Train the CNN Model
def train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)  # Move model to GPU (if available)
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Training loop
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Epoch evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {accuracy:.4f}")

    return accuracy

# Initialize the model
model = SentimentCNN(output_dim=2)

# Loss function (cross-entropy loss for binary classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model on GPU
num_epochs = 10
print("Training CNN Model...")
accuracy = train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=num_epochs)
# Accuracy of Binary Classification using Google Word2 Vec Pretrained Embedding !
print(f"Final Accuracy: {accuracy:.4f}")

Training CNN Model...
Epoch 1/10 | Loss: 0.3285 | Accuracy: 0.8851
Epoch 2/10 | Loss: 0.2675 | Accuracy: 0.8889
Epoch 3/10 | Loss: 0.2461 | Accuracy: 0.8900
Epoch 4/10 | Loss: 0.2303 | Accuracy: 0.8891
Epoch 5/10 | Loss: 0.2162 | Accuracy: 0.8869
Epoch 6/10 | Loss: 0.2031 | Accuracy: 0.8863
Epoch 7/10 | Loss: 0.1931 | Accuracy: 0.8861
Epoch 8/10 | Loss: 0.1823 | Accuracy: 0.8842
Epoch 9/10 | Loss: 0.1732 | Accuracy: 0.8842
Epoch 10/10 | Loss: 0.1640 | Accuracy: 0.8810
Final Accuracy: 0.8810


Accuracy of

In [25]:
del df

In [27]:
df_3classes

,star_rating,review_body,sentiment,cleaned_review,word_embeddings2
0,1,This item was absolutely terrible. The ink bar...,1,"[this, item, was, absolutely, terrible, the, i...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
1,1,out from the box there is a blank area when it...,1,"[out, from, the, box, there, is, blank, area, ...","[[0.110839844, -0.15820312, 0.020751953, 0.166..."
2,1,I read ALL the reviews Bad & Good and made my ...,1,"[i, read, all, the, reviews, bad, good, made, ...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
3,1,This came very sharp and it would not come apa...,1,"[this, came, very, sharp, it, would, not, come...","[[0.109375, 0.140625, -0.03173828, 0.16601562,..."
4,1,I'd give this day timer zero stars if that was...,1,"[id, give, this, day, timer, zero, stars, if, ...","[[0.29492188, -0.12109375, 0.0006828308, 0.220..."
...,...,...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,0,"[lo, que, para, mi, oficina, en, casa, muy, bu...","[[0.13769531, -0.023803711, 0.029663086, 0.197..."
249996,5,I have had Panasonic 3 phones for a long time....,0,"[i, have, had, panasonic, phones, for, long, t...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."
249997,5,These are great pens to draw with and I like t...,0,"[these, are, great, pens, draw, with, i, like,...","[[-0.016235352, 0.09423828, 0.091796875, 0.119..."
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",0,"[i, do, martial, arts, jiujitsu, i, like, take...","[[-0.22558594, -0.01953125, 0.09082031, 0.2373..."


In [28]:
df_3classes = df_3classes[["word_embeddings2", "sentiment"]]


In [29]:
del X
del y

# Split the dataset into training and testing
del X_train
del X_test
del y_train
del y_test

# Create Dataset instances
del train_dataset
del test_dataset
 #= DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for batching with smaller batch size
del batch_size #= 16  # Decrease the batch size to reduce memory usage
del train_loader #= DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
del test_loader #= DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [30]:
del model #= SentimentCNN(output_dim=2)

# Loss function (cross-entropy loss for binary classification)
del criterion #= nn.CrossEntropyLoss()

# Optimizer (Adam)
del optimizer #= optim.Adam(model.parameters(), lr=0.001)

# Train the model on GPU
del num_epochs #= 10
#print("Training CNN Model...")
del accuracy #= train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=num_epochs)

In [34]:
import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    if len(embeddings) >= max_length:
        return np.array(embeddings[:max_length])  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Apply padding/truncation
df_3classes["padded_embeddings"] = df_3classes["word_embeddings2"].apply(lambda x: pad_or_truncate(x, max_review_length, embedding_dim))


<ipython-input-34-184b52f9eb08>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3classes["padded_embeddings"] = df_3classes["word_embeddings2"].apply(lambda x: pad_or_truncate(x, max_review_length, embedding_dim))


In [35]:
df_3classes.shape

(250000, 3)

# Binary Classification using Custom Word2Vec Embedding :

In [1]:
import gensim.downloader as api
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Fetch Data
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')




<ipython-input-2-30d3103f6bae>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [3]:
df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [4]:
# Ensuring that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

df['sentiment'] = df['star_rating'].apply(map_sentiment)
df


<ipython-input-4-6d3d452f7c73>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = df['star_rating'].astype(int)
<ipython-input-4-6d3d452f7c73>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['star_rating'].apply(map_sentiment)


,star_rating,review_body,sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
3,1,Although this was labeled as &#34;new&#34; the...,2
4,4,Gorgeous colors and easy to use,1
...,...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...,1
2640348,4,Although the Palm Pilot is thin and compact it...,1
2640349,4,This book had a lot of great content without b...,1
2640350,5,I am teaching a course in Excel and am using t...,1


In [5]:
df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


<ipython-input-5-66051e4f6321>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


In [6]:
df.shape

(250000, 3)

In [7]:
df = df[df["sentiment"] != 3]
df.shape

(200000, 3)

In [8]:
import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
df["tokens"] = df["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(df["tokens"].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0    [This, item, was, absolutely, terrible, ., The...
1    [out, from, the, box, there, is, a, blank, are...
2    [I, read, ALL, the, reviews, Bad, &, Good, and...
3    [This, came, very, sharp, and, it, would, not,...
4    [I, 'd, give, this, day, timer, zero, stars, i...
Name: tokens, dtype: object


In [9]:
from gensim.models import Word2Vec

custom_w2v = Word2Vec(
    sentences=df["tokens"],  # List of tokenized reviews
    vector_size=300,         # Word embedding size
    window=11,               # Context window size
    min_count=10,            # Minimum word frequency threshold
    workers=4                # Use multiple CPU cores for efficiency
)

# Save the trained model
custom_w2v.save("custom_word2vec.model")

In [10]:

df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df

<ipython-input-10-56b2611b6d1a>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body,sentiment,tokens
0,1,3/4s of labels actually stuck on clothes<br />...,2,"[3/4s, of, labels, actually, stuck, on, clothe..."
1,1,I have NEVER felt compelled to write an online...,2,"[I, have, NEVER, felt, compelled, to, write, a..."
2,1,Total disappointment. Did not work as advertised.,2,"[Total, disappointment, ., Did, not, work, as,..."
3,1,Arrived broken complete junk not worth the money,2,"[Arrived, broken, complete, junk, not, worth, ..."
4,1,I ordered this product and according to the de...,2,"[I, ordered, this, product, and, according, to..."
...,...,...,...,...
199995,5,Purchased the product. the product i received ...,1,"[Purchased, the, product, ., the, product, i, ..."
199996,5,We've had these phones for about six months no...,1,"[We, 've, had, these, phones, for, about, six,..."
199997,5,"Very stylish, great quality and superb usabili...",1,"[Very, stylish, ,, great, quality, and, superb..."
199998,5,"Okay, so it's not really like a tropical vacat...",1,"[Okay, ,, so, it, 's, not, really, like, a, tr..."


In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))  # Load English stopwords
custom_vocab = set(custom_w2v.wv.index_to_key)
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize words
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return [word for word in tokens if word in custom_vocab]  # Keep only words in Word2Vec vocabulary

# Apply preprocessing to all reviews
df["cleaned_review"] = df["review_body"].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
del stop_words
del custom_vocab

In [13]:
def get_word_embeddings(words, model):
     return [model[word] for word in words if word in model]  # Get embeddings for each word

#Apply function to dataset
df["word_embeddings2"] = df["cleaned_review"].apply(lambda x: get_word_embeddings(x, custom_w2v.wv))
df['sentiment'] = df['sentiment'].replace({1: 0, 2: 1})

In [14]:

import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    # Efficiently pad or truncate the embeddings without creating intermediate arrays
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    embeddings = np.array(embeddings)  # Convert to a numpy array
    if len(embeddings) >= max_length:
        return embeddings[:max_length]  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Efficiently apply padding/truncation using a list comprehension
df["padded_embeddings"] = [
    pad_or_truncate(embedding, max_review_length, embedding_dim)
    for embedding in df["word_embeddings2"]
]

In [15]:
df

,star_rating,review_body,sentiment,tokens,cleaned_review,word_embeddings2,padded_embeddings
0,1,3/4s of labels actually stuck on clothes<br />...,1,"[3/4s, of, labels, actually, stuck, on, clothe...","[labels, actually, stuck, stuck, mine]","[[1.0143617, -0.48071748, 1.5226232, -2.004721...","[[1.0143617391586304, -0.4807174801826477, 1.5..."
1,1,I have NEVER felt compelled to write an online...,1,"[I, have, NEVER, felt, compelled, to, write, a...","[never, felt, compelled, write, online, review...","[[-1.2023066, -1.617459, -1.0227412, 2.011649,...","[[-1.2023066282272339, -1.6174590587615967, -1..."
2,1,Total disappointment. Did not work as advertised.,1,"[Total, disappointment, ., Did, not, work, as,...","[total, disappointment, work, advertised]","[[-0.12268394, 1.5094472, 0.48135737, -1.19185...","[[-0.12268394231796265, 1.5094472169876099, 0...."
3,1,Arrived broken complete junk not worth the money,1,"[Arrived, broken, complete, junk, not, worth, ...","[arrived, broken, complete, junk, worth, money]","[[-0.6913202, -1.5962825, 0.948902, -0.3804980...","[[-0.6913201808929443, -1.5962824821472168, 0...."
4,1,I ordered this product and according to the de...,1,"[I, ordered, this, product, and, according, to...","[ordered, product, according, description, bat...","[[1.2315072, 1.2367036, 0.9204612, -0.60151595...","[[1.2315071821212769, 1.236703634262085, 0.920..."
...,...,...,...,...,...,...,...
199995,5,Purchased the product. the product i received ...,0,"[Purchased, the, product, ., the, product, i, ...","[purchased, product, product, received, damage...","[[0.5599474, 1.7872473, 1.366111, -0.41558185,...","[[0.5599473714828491, 1.7872473001480103, 1.36..."
199996,5,We've had these phones for about six months no...,0,"[We, 've, had, these, phones, for, about, six,...","[phones, six, months, absolutely, complaints, ...","[[0.7045218, 3.9443688, -1.0582962, -1.3539333...","[[0.7045217752456665, 3.944368839263916, -1.05..."
199997,5,"Very stylish, great quality and superb usabili...",0,"[Very, stylish, ,, great, quality, and, superb...","[stylish, great, quality, superb, usability, h...","[[0.08702214, -0.3257804, -0.14667802, -0.0214...","[[0.08702214062213898, -0.32578039169311523, -..."
199998,5,"Okay, so it's not really like a tropical vacat...",0,"[Okay, ,, so, it, 's, not, really, like, a, tr...","[okay, really, like, tropical, vacation, cubic...","[[0.5538517, -1.9684213, 0.45339838, -1.995013...","[[0.5538517236709595, -1.9684213399887085, 0.4..."


In [16]:
del custom_w2v
df = df[["padded_embeddings", "sentiment"]]

# Force garbage collection to free memory
import gc
gc.collect()


90

In [ ]:
df

In [17]:
import numpy as np
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming df['padded_embeddings'] contains the padded word embeddings of size (50, 300)
# Let's also assume that df['label'] contains the binary sentiment labels (1 or 2)

# Define a custom Dataset class to handle large datasets more efficiently
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

# Convert to numpy arrays
X = np.stack(df['padded_embeddings'].values)  # Shape: (200000, 50, 300)
y = df['sentiment'].values #- 1  # Subtract 1 to adjust labels to start from 0 (for PyTorch)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Dataset instances
train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

# Create DataLoader for batching with smaller batch size
batch_size = 32  # Decrease the batch size to reduce memory usage
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Using device: cuda


In [18]:
# Define the CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, output_dim=2):
        super(SentimentCNN, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=300, out_channels=50, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.relu1 = nn.ReLU()

        # Second Convolutional Layer
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.relu2 = nn.ReLU()

        # Calculate the size of flattened features after the convolution layers
        self.flat_features = 10 * 12  # After two max pooling operations: 50 -> 25 -> 12

        # Fully Connected Layer
        self.fc = nn.Linear(self.flat_features, output_dim)

    def forward(self, x):
        # Input shape: (batch_size, seq_length, embedding_dim)
        # Transpose to: (batch_size, embedding_dim, seq_length)
        x = x.transpose(1, 2)

        # First conv layer
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        # Second conv layer
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        # Flatten
        x = x.view(-1, self.flat_features)

        # Fully connected layer
        x = self.fc(x)
        return x


In [20]:
# Train the CNN Model
def train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)  # Move model to GPU (if available)
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Training loop
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Epoch evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {accuracy:.4f}")

    return accuracy

# Initialize the model
model = SentimentCNN(output_dim=2)

# Loss function (cross-entropy loss for binary classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model on GPU
num_epochs = 10
print("Training CNN Model...")
accuracy = train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=num_epochs)

print(f"Final Accuracy of Binary Classification using Custom Word2Vec Embedding: {accuracy:.4f}")

Training CNN Model...
Epoch 1/10 | Loss: 0.3576 | Accuracy: 0.8544
Epoch 2/10 | Loss: 0.3135 | Accuracy: 0.8662
Epoch 3/10 | Loss: 0.2959 | Accuracy: 0.8662
Epoch 4/10 | Loss: 0.2819 | Accuracy: 0.8700
Epoch 5/10 | Loss: 0.2700 | Accuracy: 0.8677
Epoch 6/10 | Loss: 0.2614 | Accuracy: 0.8669
Epoch 7/10 | Loss: 0.2525 | Accuracy: 0.8682
Epoch 8/10 | Loss: 0.2450 | Accuracy: 0.8639
Epoch 9/10 | Loss: 0.2377 | Accuracy: 0.8643
Epoch 10/10 | Loss: 0.2306 | Accuracy: 0.8649
Final Accuracy of Binary Classification using Custom Word2Vec Embedding: 0.8649


### Ternary Classification using CNN :

### Using Pretrained Google Word2Vec Embedings:

In [1]:
import gensim.downloader as api
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
wv_google = api.load("word2vec-google-news-300")


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [3]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df_3classes = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


<ipython-input-3-c76272e1e50f>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_3classes = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [4]:
df_3classes

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [5]:
# Ensuring that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df_3classes['star_rating'] = pd.to_numeric(df_3classes['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df_3classes = df_3classes.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df_3classes['star_rating'] = df_3classes['star_rating'].astype(int)
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

df_3classes['sentiment'] = df_3classes['star_rating'].apply(map_sentiment)
df_3classes

<ipython-input-5-5a2b014bbd9b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3classes['star_rating'] = df_3classes['star_rating'].astype(int)
<ipython-input-5-5a2b014bbd9b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3classes['sentiment'] = df_3classes['star_rating'].apply(map_sentiment)


,star_rating,review_body,sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
3,1,Although this was labeled as &#34;new&#34; the...,2
4,4,Gorgeous colors and easy to use,1
...,...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...,1
2640348,4,Although the Palm Pilot is thin and compact it...,1
2640349,4,This book had a lot of great content without b...,1
2640350,5,I am teaching a course in Excel and am using t...,1


In [6]:
df_3classes = df_3classes.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df_3classes

<ipython-input-6-f5983cd7d7cf>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_3classes = df_3classes.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body,sentiment
0,1,This item was absolutely terrible. The ink bar...,2
1,1,out from the box there is a blank area when it...,2
2,1,I read ALL the reviews Bad & Good and made my ...,2
3,1,This came very sharp and it would not come apa...,2
4,1,I'd give this day timer zero stars if that was...,2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,1
249996,5,I have had Panasonic 3 phones for a long time....,1
249997,5,These are great pens to draw with and I like t...,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",1


In [7]:
import nltk
nltk.download('punkt_tab')
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize words
    return [word for word in tokens if word in wv_google]  # Keep only words in Word2Vec vocabulary

# Apply preprocessing to all reviews
df_3classes["cleaned_review"] = df_3classes["review_body"].apply(preprocess_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [8]:
def get_word_embeddings(words, model):
     return [model[word] for word in words if word in model]  # Get embeddings for each word

#Apply function to dataset
df_3classes["word_embeddings2"] = df_3classes["cleaned_review"].apply(lambda x: get_word_embeddings(x, wv_google))
df_3classes['sentiment'] = df_3classes['sentiment'].replace({1: 0, 2: 1,3:2})

In [9]:
import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    # Efficiently pad or truncate the embeddings without creating intermediate arrays
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    embeddings = np.array(embeddings)  # Convert to a numpy array
    if len(embeddings) >= max_length:
        return embeddings[:max_length]  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Efficiently apply padding/truncation using a list comprehension
df_3classes["padded_embeddings"] = [
    pad_or_truncate(embedding, max_review_length, embedding_dim)
    for embedding in df_3classes["word_embeddings2"]
]

In [10]:
del wv_google


In [11]:
df_3classes = df_3classes[["padded_embeddings", "sentiment"]]

# Force garbage collection to free memory
import gc
gc.collect()


0

In [13]:
df_3classes.shape

(250000, 2)

In [13]:
df_3classes['sentiment'].unique()

array([1, 2, 0])

In [11]:
X = np.stack(df_3classes['padded_embeddings'].values)  # Shape: (200000, 50, 300)
y = df_3classes['sentiment'].values #- 1  # Subtract 1 to adjust labels to start from 0 (for PyTorch)


In [12]:
del df_3classes

In [17]:
import numpy as np
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a custom Dataset class (loading data lazily)
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings  # Store as list (avoid numpy overhead)
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        embedding = embedding.unsqueeze(0)  # Add an additional channel dimension (1 channel)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

# Avoid NumPy (huge RAM consumption), use lists
X = df_3classes['padded_embeddings'].tolist()
y = df_3classes['sentiment'].values #- 1  # Convert labels to start from 0

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create dataset instances
train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

# Create DataLoader for batching with smaller batch size
batch_size = 16  # Increase batch size if possible
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Dataset loaded with {len(train_dataset)} training and {len(test_dataset)} testing samples.")


# Define the CNN Model for sentiment analysis
class SentimentCNN(nn.Module):
    def __init__(self, output_dim):
        super(SentimentCNN, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=50, kernel_size=(3, 300), padding=(1, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1))
        self.relu1 = nn.ReLU()

        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size=(3, 1), padding=(1, 0))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 1))
        self.relu2 = nn.ReLU()

        # Calculate the size of flattened features after convolutions and pooling
        self.flat_features = 10 * 12  # After two max pooling operations

        # Fully Connected Layer
        self.fc = nn.Linear(self.flat_features, output_dim)

    def forward(self, x):
        # Input shape: (batch_size, channels, seq_length, embedding_dim)
        # First conv layer
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        # Second conv layer
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        # Flatten
        x = x.view(-1, self.flat_features)

        # Fully connected layer
        x = self.fc(x)
        return x


# Train the CNN Model
def train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)  # Move model to GPU (if available)
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Training loop
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Epoch evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {accuracy:.4f}")

    return accuracy


# Initialize the model
model = SentimentCNN(output_dim=3)  # Set output_dim based on number of classes

# Loss function (cross-entropy loss for binary classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model on GPU
num_epochs = 10
print("Training CNN Model...")
accuracy = train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=num_epochs)

# Ternary Classification Accuracy using Pretrained Google Word Embeddings
print(f"Final Accuracy: {accuracy:.4f}")


Using device: cuda
Dataset loaded with 200000 training and 50000 testing samples.
Training CNN Model...
Epoch 1/10 | Loss: 0.7062 | Accuracy: 0.7204
Epoch 2/10 | Loss: 0.6413 | Accuracy: 0.7265
Epoch 3/10 | Loss: 0.6194 | Accuracy: 0.7274
Epoch 4/10 | Loss: 0.6040 | Accuracy: 0.7281
Epoch 5/10 | Loss: 0.5908 | Accuracy: 0.7294
Epoch 6/10 | Loss: 0.5804 | Accuracy: 0.7248
Epoch 7/10 | Loss: 0.5706 | Accuracy: 0.7279
Epoch 8/10 | Loss: 0.5611 | Accuracy: 0.7278
Epoch 9/10 | Loss: 0.5532 | Accuracy: 0.7266
Epoch 10/10 | Loss: 0.5454 | Accuracy: 0.7260
Final Accuracy: 0.7260


## Using Custom Word Embeddings  :

## Ternary Classification using Custom Word Embedding :

In [1]:
import gensim.downloader as api
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Fetch Data
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df_3classes = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')



<ipython-input-2-67ac6f7cba9a>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_3classes = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [3]:
df_3classes

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [3]:
# Ensuring that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df_3classes['star_rating'] = pd.to_numeric(df_3classes['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df_3classes = df_3classes.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df_3classes['star_rating'] = df_3classes['star_rating'].astype(int)
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

df_3classes['sentiment'] = df_3classes['star_rating'].apply(map_sentiment)
df_3classes


<ipython-input-3-2db616368a4e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3classes['star_rating'] = df_3classes['star_rating'].astype(int)


,star_rating,review_body,sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
3,1,Although this was labeled as &#34;new&#34; the...,2
4,4,Gorgeous colors and easy to use,1
...,...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...,1
2640348,4,Although the Palm Pilot is thin and compact it...,1
2640349,4,This book had a lot of great content without b...,1
2640350,5,I am teaching a course in Excel and am using t...,1


In [4]:
df_3classes = df_3classes.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df_3classes

<ipython-input-4-f5983cd7d7cf>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_3classes = df_3classes.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body,sentiment
0,1,This item was absolutely terrible. The ink bar...,2
1,1,out from the box there is a blank area when it...,2
2,1,I read ALL the reviews Bad & Good and made my ...,2
3,1,This came very sharp and it would not come apa...,2
4,1,I'd give this day timer zero stars if that was...,2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUE...,1
249996,5,I have had Panasonic 3 phones for a long time....,1
249997,5,These are great pens to draw with and I like t...,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to tak...",1


Tokenizing :

In [5]:

import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
df_3classes["tokens"] = df_3classes["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(df_3classes["tokens"].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0    [This, item, was, absolutely, terrible, ., The...
1    [out, from, the, box, there, is, a, blank, are...
2    [I, read, ALL, the, reviews, Bad, &, Good, and...
3    [This, came, very, sharp, and, it, would, not,...
4    [I, 'd, give, this, day, timer, zero, stars, i...
Name: tokens, dtype: object


In [6]:
from gensim.models import Word2Vec

custom_w2v = Word2Vec(
    sentences=df_3classes["tokens"],  # List of tokenized reviews
    vector_size=300,         # Word embedding size
    window=11,               # Context window size
    min_count=10,            # Minimum word frequency threshold
    workers=4                # Use multiple CPU cores for efficiency
)

# Save the trained model
custom_w2v.save("custom_word2vec.model")

In [7]:
df_3classes = df_3classes.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
df_3classes

<ipython-input-7-f5983cd7d7cf>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_3classes = df_3classes.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body,sentiment,tokens
0,1,3/4s of labels actually stuck on clothes<br />...,2,"[3/4s, of, labels, actually, stuck, on, clothe..."
1,1,I have NEVER felt compelled to write an online...,2,"[I, have, NEVER, felt, compelled, to, write, a..."
2,1,Total disappointment. Did not work as advertised.,2,"[Total, disappointment, ., Did, not, work, as,..."
3,1,Arrived broken complete junk not worth the money,2,"[Arrived, broken, complete, junk, not, worth, ..."
4,1,I ordered this product and according to the de...,2,"[I, ordered, this, product, and, according, to..."
...,...,...,...,...
249995,5,Purchased the product. the product i received ...,1,"[Purchased, the, product, ., the, product, i, ..."
249996,5,We've had these phones for about six months no...,1,"[We, 've, had, these, phones, for, about, six,..."
249997,5,"Very stylish, great quality and superb usabili...",1,"[Very, stylish, ,, great, quality, and, superb..."
249998,5,"Okay, so it's not really like a tropical vacat...",1,"[Okay, ,, so, it, 's, not, really, like, a, tr..."


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))  # Load English stopwords
custom_vocab = set(custom_w2v.wv.index_to_key)
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize words
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return [word for word in tokens if word in custom_vocab]  # Keep only words in Word2Vec vocabulary

# Apply preprocessing to all reviews
df_3classes["cleaned_review"] = df_3classes["review_body"].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
del stop_words
del custom_vocab

In [10]:
def get_word_embeddings(words, model):
     return [model[word] for word in words if word in model]  # Get embeddings for each word

#Apply function to dataset
df_3classes["word_embeddings2"] = df_3classes["cleaned_review"].apply(lambda x: get_word_embeddings(x, custom_w2v.wv))
df_3classes['sentiment'] = df_3classes['sentiment'].replace({1: 0, 2: 1,3:2})

In [11]:
import numpy as np

max_review_length = 50  # Fixed review length
embedding_dim = 300  # Word2Vec vector size

def pad_or_truncate(embeddings, max_length=50, vector_size=300):
    # Efficiently pad or truncate the embeddings without creating intermediate arrays
    if len(embeddings) == 0:
        return np.zeros((max_length, vector_size))  # Handle empty reviews properly

    embeddings = np.array(embeddings)  # Convert to a numpy array
    if len(embeddings) >= max_length:
        return embeddings[:max_length]  # Truncate if too long

    # Pad if too short
    padding = np.zeros((max_length - len(embeddings), vector_size))
    return np.vstack((embeddings, padding))

# Efficiently apply padding/truncation using a list comprehension
df_3classes["padded_embeddings"] = [
    pad_or_truncate(embedding, max_review_length, embedding_dim)
    for embedding in df_3classes["word_embeddings2"]
]


In [18]:
df_3classes

,star_rating,review_body,sentiment,tokens,cleaned_review,word_embeddings2,padded_embeddings
0,1,3/4s of labels actually stuck on clothes<br />...,1,"[3/4s, of, labels, actually, stuck, on, clothe...","[labels, actually, stuck, stuck, mine]","[[1.965702, 1.0855994, -1.2386054, -2.9874034,...","[[1.9657020568847656, 1.085599422454834, -1.23..."
1,1,I have NEVER felt compelled to write an online...,1,"[I, have, NEVER, felt, compelled, to, write, a...","[never, felt, compelled, write, online, review...","[[-1.1844839, 1.6098195, -0.9391377, -2.092492...","[[-1.1844838857650757, 1.6098195314407349, -0...."
2,1,Total disappointment. Did not work as advertised.,1,"[Total, disappointment, ., Did, not, work, as,...","[total, disappointment, work, advertised]","[[0.66259134, 1.1419787, -0.6302726, 0.4839093...","[[0.6625913381576538, 1.1419787406921387, -0.6..."
3,1,Arrived broken complete junk not worth the money,1,"[Arrived, broken, complete, junk, not, worth, ...","[arrived, broken, complete, junk, worth, money]","[[1.7550042, 0.7391219, 1.3830007, 1.2199185, ...","[[1.7550041675567627, 0.7391219139099121, 1.38..."
4,1,I ordered this product and according to the de...,1,"[I, ordered, this, product, and, according, to...","[ordered, product, according, description, bat...","[[-0.74498326, 2.2431676, -0.07507067, 0.51059...","[[-0.7449832558631897, 2.2431676387786865, -0...."
...,...,...,...,...,...,...,...
249995,5,Purchased the product. the product i received ...,0,"[Purchased, the, product, ., the, product, i, ...","[purchased, product, product, received, damage...","[[-0.36651203, 0.68778765, 1.1802238, 1.950325...","[[-0.3665120303630829, 0.687787652015686, 1.18..."
249996,5,We've had these phones for about six months no...,0,"[We, 've, had, these, phones, for, about, six,...","[phones, six, months, absolutely, complaints, ...","[[1.2051318, 2.4352815, -1.0352412, -1.4581057...","[[1.2051317691802979, 2.43528151512146, -1.035..."
249997,5,"Very stylish, great quality and superb usabili...",0,"[Very, stylish, ,, great, quality, and, superb...","[stylish, great, quality, superb, usability, h...","[[-0.30489233, -0.92279, -0.33035713, -1.16818...","[[-0.30489233136177063, -0.922789990901947, -0..."
249998,5,"Okay, so it's not really like a tropical vacat...",0,"[Okay, ,, so, it, 's, not, really, like, a, tr...","[okay, really, like, tropical, vacation, cubic...","[[1.1527342, -0.04992801, -0.06777622, -1.1586...","[[1.1527341604232788, -0.04992800951004028, -0..."


In [12]:
del custom_w2v
df_3classes = df_3classes[["padded_embeddings", "sentiment"]]

# Force garbage collection to free memory
import gc
gc.collect()


0

In [17]:
df_3classes.shape


(250000, 2)

In [18]:
# df_3classes['sentiment'].unique()
# X = np.stack(df_3classes['padded_embeddings'].values)  # Shape: (200000, 50, 300)
# y = df_3classes['sentiment'].values #- 1  # Subtract 1 to adjust labels to start from 0 (for PyTorch)
# del df_3classes


In [19]:
import gc

#del df['tokens']  # Delete the column
gc.collect()

0

In [13]:
import numpy as np
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define a custom Dataset class (loading data lazily)
class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings  # Store as list (avoid numpy overhead)
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float32)
        embedding = embedding.unsqueeze(0)  # Add an additional channel dimension (1 channel)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

# Avoid NumPy (huge RAM consumption), use lists
X = df_3classes['padded_embeddings'].tolist()
y = df_3classes['sentiment'].values #- 1  # Convert labels to start from 0

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create dataset instances
train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

# Create DataLoader for batching with smaller batch size
batch_size = 32  # Increase batch size if possible
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Dataset loaded with {len(train_dataset)} training and {len(test_dataset)} testing samples.")


# Define the CNN Model for sentiment analysis
class SentimentCNN(nn.Module):
    def __init__(self, output_dim):
        super(SentimentCNN, self).__init__()

        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=50, kernel_size=(3, 300), padding=(1, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 1))
        self.relu1 = nn.ReLU()

        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=10, kernel_size=(3, 1), padding=(1, 0))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 1))
        self.relu2 = nn.ReLU()

        # Calculate the size of flattened features after convolutions and pooling
        self.flat_features = 10 * 12  # After two max pooling operations

        # Fully Connected Layer
        self.fc = nn.Linear(self.flat_features, output_dim)

    def forward(self, x):
        # Input shape: (batch_size, channels, seq_length, embedding_dim)
        # First conv layer
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        # Second conv layer
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        # Flatten
        x = x.view(-1, self.flat_features)

        # Fully connected layer
        x = self.fc(x)
        return x


# Train the CNN Model
def train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)  # Move model to GPU (if available)
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Training loop
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Epoch evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move batch data to GPU
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {accuracy:.4f}")

    return accuracy


# Initialize the model
model = SentimentCNN(output_dim=3)  # Set output_dim based on number of classes

# Loss function (cross-entropy loss for binary classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model on GPU
num_epochs = 10
print("Training CNN Model...")
accuracy = train_cnn(model, train_loader, test_loader, criterion, optimizer, epochs=num_epochs)

print(f"Final Accuracy: {accuracy:.4f}")



Using device: cuda
Dataset loaded with 200000 training and 50000 testing samples.
Training CNN Model...
Epoch 1/10 | Loss: 0.7387 | Accuracy: 0.7013
Epoch 2/10 | Loss: 0.6928 | Accuracy: 0.7046
Epoch 3/10 | Loss: 0.6752 | Accuracy: 0.7062
Epoch 4/10 | Loss: 0.6624 | Accuracy: 0.7065
Epoch 5/10 | Loss: 0.6522 | Accuracy: 0.7102
Epoch 6/10 | Loss: 0.6433 | Accuracy: 0.7094
Epoch 7/10 | Loss: 0.6356 | Accuracy: 0.7089
Epoch 8/10 | Loss: 0.6283 | Accuracy: 0.7069
Epoch 9/10 | Loss: 0.6219 | Accuracy: 0.7034
Epoch 10/10 | Loss: 0.6155 | Accuracy: 0.7036
Final Accuracy: 0.7036
